In [ ]:
!pip install -qqq glove-python3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.0/327.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
from glove import Corpus, Glove
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from sklearn.preprocessing import LabelEncoder
import nltk

nltk.download('punkt')

# Sample messages dataset (toy dataset for beginners)
messages = [
    "Win a free iPhone now!",          # spam
    "Congratulations, you won a prize!", # spam
    "Meeting at 5 PM today",           # ham
    "Let’s have lunch tomorrow",       # ham
    "You won a lottery ticket!",       # spam
    "Project discussion at 3 PM",      # ham
    "Exclusive offer, buy now!",       # spam
    "Join us for a meeting tomorrow"   # ham
]

labels = ["spam", "spam", "ham", "ham", "spam", "ham", "spam", "ham"]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Tokenize the messages
tokenized_messages = [word_tokenize(msg.lower()) for msg in messages]

# Create a GloVe corpus and generate a co-occurrence matrix
corpus = Corpus()
corpus.fit(tokenized_messages, window=3)  # Adjust window size if needed

# Train GloVe model
glove = Glove(no_components=50, learning_rate=0.05)  # 50-dimensional embeddings
glove.fit(corpus.matrix, epochs=10, no_threads=2, verbose=True)
glove.add_dictionary(corpus.dictionary)

# Create a word embedding matrix
vocab_size = len(glove.dictionary)
embedding_dim = glove.no_components

# Map words to GloVe embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in glove.dictionary.items():
    embedding_matrix[idx] = glove.word_vectors[idx]


Performing 10 training epochs with 2 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [ ]:
# Convert sentences to sequences of indices
def encode_message(msg, dictionary, max_length=10):
    tokens = word_tokenize(msg.lower())
    return [dictionary.get(word, 0) for word in tokens][:max_length]

# Define maximum message length
max_length = 10
encoded_messages = [encode_message(msg, glove.dictionary, max_length) for msg in messages]

# Pad sequences to uniform length
X = np.array([seq + [0] * (max_length - len(seq)) for seq in encoded_messages])

# Convert labels to binary (0 = ham, 1 = spam)
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
import tensorflow as tf

# Build the neural network model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim,
              weights=[embedding_matrix], input_length=max_length, trainable=False),
    Flatten(),
    Dense(16, activation='relu'),  # Hidden layer with 16 neurons
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4500 - loss: 0.6949
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4250 - loss: 0.6920     
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7125 - loss: 0.6897 
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8375 - loss: 0.6882 
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8375 - loss: 0.6874 
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8375 - loss: 0.6857 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8375 - loss: 0.6859 
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8375 - loss: 0.6847 
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8375 - loss: 0.6836 
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6750 - loss: 0.6801 
Test Accuracy: 0.33


In [ ]:
# Predict on new messages
new_messages = ["You won a free car!", "Meeting tomorrow at 10 AM"]
encoded_new = [encode_message(msg, glove.dictionary, max_length) for msg in new_messages]
X_new = np.array([seq + [0] * (max_length - len(seq)) for seq in encoded_new])

predictions = model.predict(X_new)
for msg, pred in zip(new_messages, predictions):
    print(f"Message: '{msg}' -> {'Spam' if pred > 0.5 else 'Ham'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Message: 'You won a free car!' -> Ham
Message: 'Meeting tomorrow at 10 AM' -> Ham
